# Patient Administrative Outcomes Predictive Model

# 00. Setup Development Environment

- Install Poetry to simplify dependency management.
    - `pip install poetry`
- Setup Poetery Project
    - `poetry init`
- Create a virtual envrionment
    - `python -m venv .venv`
- Add dependencies 
    - `poetry add pyspark ipykernel pandas matplotlib ploty`


In [1]:
import os
import pandas as pd
import json
import matplotlib.pyplot as plt

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark = SparkSession.builder \
    .appName("FhirDataApplication") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.local.type", "hadoop") \
    .config("spark.sql.catalog.local.warehouse", "dataset/iceberg") \
    .getOrCreate()

# 01. Data Integration

**FHIR Resources**
- Patient: For demographic information.
- Encounter: To track patient visits and interactions.
- Condition: For diagnoses and health conditions.
- Procedure: For medical procedures performed.
- Observation: For lab results and vital signs

## Bronze Layer

In [ ]:
spark.sql("""
  CREATE TABLE IF NOT EXISTS local.bronze.resource (
    sourceFile STRING,
    value STRING
  ) USING iceberg;
""")

spark.sql("""
SELECT sourceFile, count(value)
FROM local.bronze.resource GROUP BY sourceFile
""").show(10, truncate=False)

In [ ]:
RESOURCE_FILES = ["MimicPatient.ndjson.gz", "MimicCondition.ndjson.gz", "MimicConditionED.ndjson.gz", "MimicEncounter.ndjson.gz", "MimicEncounterED.ndjson.gz", "MimicEncounterICU.ndjson.gz"]
for resource_file in RESOURCE_FILES:
    resource_df =  spark.read.text(f"dataset/{resource_file}")
    resource_df = resource_df.withColumn("sourceFile", F.lit(resource_file))
    resource_df.write.format("iceberg").mode("append").save("local.bronze.resource")

## Silver Layer

In [22]:
raw_df = spark.read.format("iceberg").load("local.bronze.resource")

### Patient FM

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS local.silver.patient (
    patientId        STRING,
    gender           STRING,
    birthDate        DATE,
    maritalStatus    STRING,
    sourceFile      STRING
) USING iceberg;
""")

In [ ]:
patient_df = raw_df.filter(raw_df.sourceFile.isin("MimicPatient.ndjson.gz"))
with open("schema/Patient.json") as f:
    schema_read = json.loads(f.read())
schema =  T.StructType.fromJson(schema_read)
patient_df = patient_df.withColumn("parsed_json", F.from_json(patient_df["value"], schema))
patient_df = patient_df.select("parsed_json.*", "sourceFile")
patient_df.createOrReplaceTempView("patient_df")

fm_patient = spark.sql("""
select 
    id AS patientId,
    gender AS gender,
    to_date(birthDate) birthDate,
    CASE maritalStatus.coding[0].code
        WHEN "M" THEN "Married"
        WHEN "D" THEN "Divorced"
        WHEN "W" THEN "Widowed"
        WHEN "S" THEN "Never Married"
        ELSE "Unknown"
    END maritalStatus,
    sourceFile
from patient_df
""")

fm_patient.show(5, truncate=False)
fm_patient.write.format("iceberg").mode("overwrite").save("local.silver.patient")

### Encounter FM

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS local.silver.encounter (
    encounterId          STRING,
    patientId            STRING,
    ref_encounterId      STRING,
    periodStart          TIMESTAMP,
    periodEnd            TIMESTAMP,
    duration             INT,
    status               STRING,
    encounterClass       STRING,
    displayType          STRING,
    admitSource          STRING,
    dischargeDisposition STRING,
    priority             STRING,
    nextEncounterId      STRING,
    readmissionStatus    STRING,
    sourceFile           STRING
) USING iceberg;
""")

In [ ]:
encounter_df = raw_df.filter(raw_df.sourceFile.isin("MimicEncounter.ndjson.gz", "MimicEncounterED.ndjson.gz"))
with open("schema/Encounter.json") as f:
    schema_read = json.loads(f.read())
schema =  T.StructType.fromJson(schema_read)
encounter_df = encounter_df.withColumn("parsed_json", F.from_json(encounter_df["value"], schema))
encounter_df = encounter_df.select("parsed_json.*", "sourceFile")
encounter_df.createOrReplaceTempView("encounter_df")

fm_encounter = spark.sql("""
SELECT
    id AS encounterId,
    replace(subject.reference, "Patient/", "") AS patientId,
    replace(partOf.reference, "Encounter/", "") AS ref_encounterId,
    CAST(period.start AS timestamp) periodStart,
    CAST(period.end AS timestamp) periodEnd,
    date_diff(day, periodStart, periodEnd) duration,
    status AS status,
    CASE class.code 
        WHEN "AMB" THEN "Ambulatory"
        WHEN "OBSENC" THEN "Observation Encounter"
        WHEN "ACUTE" THEN "Inpatient Acute"
        WHEN "EMER" THEN "Emergency"
        WHEN "SS" THEN "Short Stay"
    END AS encounterClass, 
    type[0].coding[0].display AS displayType,
    hospitalization.admitSource.coding[0].code admitSource,
    hospitalization.dischargeDisposition.coding[0].code dischargeDisposition,
    nvl(priority.coding[0].display, 'emergency') AS priority,
    -- Next EncounterID & Readmission Status
    LEAD(id) OVER (PARTITION BY subject.reference ORDER BY period.start) AS nextEncounterId,
    CASE 
        WHEN DATEDIFF(day, period.end, LEAD(period.start) OVER (PARTITION BY subject.reference ORDER BY period.start)) <= 30 THEN 'Readmission'
        ELSE 'No Readmission'
    END AS readmissionStatus,
    sourceFile
FROM encounter_df
""")

fm_encounter.show(5, truncate=False)
fm_encounter.write.format("iceberg").mode("overwrite").save("local.silver.encounter")

### Condition FM

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS local.silver.condition (
    conditionId         STRING,
    patientId           STRING,
    encounterId         STRING,
    categoryCode        STRING,
    conditionCode       STRING,
    conditionDisplay    STRING,
    conditionSystem     STRING,
    sourceFile          STRING
) USING iceberg;
""")

In [ ]:
condition_df = raw_df.filter(raw_df.sourceFile.isin("MimicCondition.ndjson.gz", "MimicConditionED.ndjson.gz"))
with open("schema/Condition.json") as f:
    schema_read = json.loads(f.read())
schema =  T.StructType.fromJson(schema_read)
condition_df = condition_df.withColumn("parsed_json", F.from_json(condition_df["value"], schema))
condition_df = condition_df.select("parsed_json.*", "sourceFile")
condition_df.createOrReplaceTempView("condition_df")

fm_condition = spark.sql("""
SELECT
    id AS conditionId,
    replace(subject.reference, "Patient/", "") AS patientId,
    replace(encounter.reference, "Encounter/", "") AS encounterId,
    category[0].coding[0].code categoryCode,
    code.coding[0].code AS conditionCode,
    code.coding[0].display AS conditionDisplay,
    code.coding[0].system AS conditionSystem,
    sourceFile
FROM condition_df
""")

fm_condition.show(5, truncate=False)
fm_condition.write.format("iceberg").mode("overwrite").save("local.silver.condition")

# 02. Exploratory Data Analysis

In [3]:
# !pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Patient Domain

- `patientID`: Unique identifier for the patient
- `gender`: Gender of the patient (male/female)
- `birthDate`: Birth Date
- `maritalStatus`: Marital Status 

In [ ]:
px.line(
    spark.sql("select date_format(birthDate, 'yyyy') birthYear, count(*) count  from local.silver.patient group by 1 order by 1").toPandas(), 
    x='birthYear', 
    y='count', 
    title='Patient BirthYear Distribution [De-Identified]'
).update_layout(
    xaxis_title='Birth Year',
    yaxis_title='Number of Patients',
    template='plotly_white'  
).show()

In [ ]:
px.pie(
    spark.sql("select gender, count(*) count  from local.silver.patient group by 1 order by 1").toPandas(),
    values='count', names='gender', title='Gender Distribution'
).show()

In [ ]:
px.bar(
    spark.sql("select gender, maritalStatus, count(*) count  from local.silver.patient group by 1,2").toPandas(), 
    x='maritalStatus', 
    y='count', 
    color='gender', 
    title='Count by Gender and Marital Status', 
    barmode='group'
).show()

## Encounter Domain

The Encounter is a resource that represents an interaction between a patient and healthcare provider(s) for the purpose of providing healthcare services or assessing the patient's health status.

It records the full span of a hospital stay, including admission, stay and discharge. It includes details such as admission start and end time, context for the admission and patient movements within the hospital.


**Table Attributes**

- `encounterId`: Unique identifier for the encounter.
- `patientId`: Unique identifier for the patient.
- `ref_encounterId`:Reference to a related encounter; can reference both future and past encounters.
- `periodStart`: Start timestamp of the encounter period.
- `periodEnd`:End timestamp of the encounter period.
- `duration`: Total duration of encounter
- `status`: Current status of the encounter (e.g., planned, in-progress, finished).
- `encounterClass`: Classification of the encounter (e.g., inpatient, outpatient); Helps categorize the nature of the healthcare service provided.
- `codedType`: Code representing the specific type of encounter.
- `displayType`: Display name for the type of encounter.
- `systemType`: System from which the type code is derived.
- `priority`: Urgency of the encounter such as routine, urgent, or emergency; 

In [ ]:
px.line(
    spark.sql("""
    select date_format(periodStart, 'yyyy') encounterYear, count(*) count  from local.silver.encounter group by 1 order by 1 
    """).toPandas(), 
    x='encounterYear', 
    y='count', 
    title='Encounter Period Year Distribution [De-Identified]'
).update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Encounters',
    template='plotly_white'  
).show()

In [ ]:
px.pie(
    spark.sql("select sourceFile, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='sourceFile', title='Source Distribution').show()

In [ ]:
px.pie(
    spark.sql("select encounterClass, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='encounterClass', title='Encounter Class Distribution').show()

In [ ]:
px.pie(
    spark.sql("select priority, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='priority', title='Encounter Class by Priority').show()

In [ ]:
px.pie(
    spark.sql("select readmissionStatus, count(*) count from local.silver.encounter group by 1").toPandas(),
values='count', names='readmissionStatus', title='Patient Readmission').show()

In [ ]:
px.line(
    spark.sql("""select readmissionStatus, year(periodStart) year, count(*) count
    from local.silver.encounter group by 1,2 order by 2""").toPandas(), 
    x='year', 
    y='count', 
    color='readmissionStatus',  # Differentiate lines by readmissionStatus
    title='Encounter Over the Years by ReadmissionStatus', 
    labels={'year': 'Year', 'count': 'Number of Encounters'}).update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Encounters',
    legend_title='Readmission Status',
    template='plotly_white'  
).show()

In [ ]:
priority_encounter_df = spark.sql("""
SELECT
    e1.priority priority1,
    e2.priority priority2,
    count(*) count
FROM local.silver.encounter e1
JOIN local.silver.encounter e2
ON e1.nextEncounterId = e2.encounterID
WHERE e1.readmissionStatus == 'Readmission'
GROUP BY all
ORDER by 3 desc
""")

priority_encounter_pd = priority_encounter_df.toPandas()
all_priorities = list(set(priority_encounter_pd['priority1'].tolist() + 
                          priority_encounter_pd['priority2'].tolist()))
node_map = {priority: idx for idx, priority in enumerate(all_priorities)}
priority_encounter_pd['source'] = priority_encounter_pd['priority1'].map(node_map)
priority_encounter_pd['target'] = priority_encounter_pd['priority2'].map(node_map)

table_trace = go.Table(
    header=dict(values=["Priority 1", "Priority 2", "Count"], fill_color='lightgrey', align='center'),
    cells=dict(values=[priority_encounter_pd['priority1'], priority_encounter_pd['priority2'], priority_encounter_pd['count']],
               fill_color='white', align='center')
)

sankey_trace = go.Sankey(
    node=dict(
        pad=15,  # Padding between nodes
        thickness=20,  # Node thickness
        line=dict(color="black", width=0.5),  # Node border settings
        label=all_priorities  # Node labels
    ),
    link=dict(
        source=priority_encounter_pd['source'],  # Source nodes (indices)
        target=priority_encounter_pd['target'],  # Target nodes (indices)
        value=priority_encounter_pd['count']     # Flow values (counts)
    )
)

fig = make_subplots(
    rows=1, cols=2, 
    column_widths=[0.7, 0.3],  # Adjust column widths (30% table, 70% Sankey)
    specs=[[{"type": "table"}, {"type": "sankey"}]],  # Specify types for each subplot
)


fig.add_trace(sankey_trace, row=1, col=1)
fig.add_trace(table_trace, row=1, col=2)

fig.update_layout(
    title_text="Readmission Encounters: Priority Transitions Sankey Diagram and Table",
    font_size=12,
    height=500
)
fig.show()

In [ ]:
spark.sql("""
SELECT 
    sourceFile, 
    CASE 
        WHEN duration BETWEEN 0 AND 5 THEN '0 to 5 days'
        WHEN duration BETWEEN 6 AND 10 THEN '6 to 10 days'
        WHEN duration BETWEEN 11 AND 20 THEN '11 to 20 days'
        WHEN duration BETWEEN 21 AND 30 THEN '21 to 30 days'
        WHEN duration BETWEEN 31 AND 50 THEN '31 to 50 days'
        WHEN duration BETWEEN 51 AND 100 THEN '51 to 100 days'
        WHEN duration BETWEEN 101 AND 150 THEN '101 to 150 days'
        WHEN duration BETWEEN 151 AND 200 THEN '151 to 200 days'
        WHEN duration BETWEEN 201 AND 250 THEN '201 to 250 days'
        WHEN duration BETWEEN 251 AND 300 THEN '251 to 300 days'
        ELSE 'More than 300 days'
    END AS duration_group,
    COUNT(*) AS encounter_count
FROM local.silver.encounter
GROUP BY sourceFile, duration_group
ORDER BY sourceFile, double(split_part(duration_group, ' ', 1));
""").show(50, truncate = False)

In [ ]:
px.line(
    spark.sql("""SELECT 
        sourceFile,
        duration AS duration_group,
        COUNT(*) AS encounter_count
    FROM local.silver.encounter
    where duration BETWEEN 1 AND 30
    GROUP BY sourceFile, duration_group
    ORDER BY sourceFile, duration_group;""").toPandas(),
    x='duration_group', y='encounter_count', color='sourceFile',
    markers=True, title="Encounter Count by Source and Duration Group").show()

## Condition Domain

The Condition resource in FHIR is used to record detailed information about a patient’s health state, including diagnoses, problems, or other clinical concerns. Here are the key points:

- Scope and Usage: It captures conditions that have risen to a level of concern, such as diseases, health issues, or post-procedure states.
- Clinical Context: Conditions can be recorded based on a clinician’s assessment or expressed by the patient or care team members.
- Examples: Conditions like pregnancy, post-surgical states, or chronic illnesses can be documented. It can also include social determinants of health like unemployment or lack of transportation.

Data Preprocessing Condition Code System is both ICD 9 & ICD 10, Standardize ICD-9 and ICD-10 codes to a common standard. GEMs (General Equivalence Mappings) are crosswalks between ICD-9 and ICD-10 codes. They help map codes from ICD-9-CM to ICD-10-CM and vice versa.

https://www.cms.gov/Medicare/Coding/ICD10/Downloads/ICD-10_GEM_fact_sheet.pdf

**Table Attributes**

- `conditionId`: The unique identifier for the condition.
- `patientId`: The unique identifier for the patient.
- `encounterId`: The unique identifier for the encounter.
- `categoryCode`: Condition category.
- `conditionCode`: The code representing the specific condition.
- `conditionDisplay`: The display name for the condition.
- `conditionSystem`: The system from which the condition code is derived.

In [ ]:
px.pie(
    spark.sql("select split_part(conditionSystem, '/', -1) conditionSystem, count(*) count from local.silver.condition group by 1").toPandas(),
    values='count', names='conditionSystem', title='Condition System Count Distribution').show()

Based on the `Condition System Count Distribution` chart, the dataset incorporates both ICD-9 and ICD-10 coding systems in condition table.

The difference between these two coding systems can introduce challenges which may affect the model performance and interpretability.

1. Inconsistent Coding:
    - ICD-10 has more granularity in specifying types of diseases, injury location, and severity.
    - ICD-9 has fewer and less specific codes than ICD-10, which means that a single ICD-9 code could map to multiple ICD-10 codes.
    - This inconsistency can create noisy features in your dataset if the same condition is coded differently depending on the coding system used. This could confuse your model, leading to reduced predictive accuracy.
2. Feature Engineering Complexity:
    - ICD-9 and ICD-10 are structured differently, both in terms of the number of codes and their specificity.
    - This complicates the creation of features related to diagnosis categories or comorbidities.
    - The features might not capture the full clinical picture, potentially leading to underfitting or overfitting
3. Data Heterogeneity:
    - Introduction of temporal bias into the model
    - This could cause the model to overestimate or underestimate readmission risks if it correlates newer coding systems with better or worse outcomes.
4. Model Interpretability:
    - Mixed coding systems make model interpretation harder, especially if using interpretable models like decision trees or logistic regression.
    - This may end up with features that are not comparable between ICD-9 and ICD-10, complicating efforts to explain your model's predictions.

In [ ]:
spark.sql("""
select conditionCode, conditionDisplay, count(*) count from local.silver.condition
where conditionSystem like '%mimic-diagnosis-icd10'
group by all order by 3 desc 
""").show(10, truncate = False)

In [ ]:
spark.sql("""
select conditionCode, conditionDisplay, count(*) count from local.silver.condition
where conditionSystem like '%mimic-diagnosis-icd9'
group by all order by 3 desc 
""").show(10, truncate = False)

Comparing top condition for both coding systems, there are few notable common types between them.

For Example:

- ICD 9 Code 4019 `Unspecified essential hypertension` is similar to ICD 10 Code I10 `Essential (primary) hypertension`.
- ICD 9 Code V1582 `Personal history of tobacco use` is similar to ICD 10 Code Z87891 `Personal history of nicotine dependence`.
- ICD 9 Code 2724 `Other and unspecified hyperlipidemia	` is similar to ICD 10 Code E785 `Hyperlipidemia, unspecified`.

The dataset must be standardized in order to gain better accuracy of predictive model.

# 03. Feature Engineering

## Condition Domain

### Standardarize Coding System

One solution is to map ICD-9 codes to ICD-10 equivalents using tools like the General Equivalence Mappings (GEMs). This allows you to convert ICD-9 codes to ICD-10 to standardize the dataset.

- Dataset Link: [ICD-9-CM to and from ICD-10-CM and ICD-10-PCS Crosswalk or General Equivalence Mappings](https://www.nber.org/research/data/icd-9-cm-and-icd-10-cm-and-icd-10-pcs-crosswalk-or-general-equivalence-mappings)

**ICD-10 Code Structure**
- Characters 1:3 = Indicate the category of the diagnosis
- Characters 4:6 = Indicate etiology, anatomic site, severity or other clinical detail
- Character 7 = Extension

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS local.silver.condition_standard (
    conditionId         STRING,
    encounterId         STRING,
    patientId           STRING,
    conditionCode       STRING
) USING iceberg;
""")

In [ ]:
icd10_df = spark.sql("""
    SELECT
        conditionId, encounterId, patientId, conditionCode
    FROM local.silver.condition condition 
    WHERE condition.conditionSystem LIKE '%icd10'
""")
icd10_df.write.format("iceberg").mode("append").save("local.silver.condition_standard")

In [6]:
df = spark.read.csv("dataset/icd9toicd10cmgem.csv", header = True)
df.createOrReplaceTempView("GemMapping")

icd9_df = spark.sql("""
    SELECT
        condition.conditionId,
        condition.encounterId,
        condition.patientId,
        GemMapping.icd10cm conditionCode
    FROM local.silver.condition condition
    JOIN GemMapping
    ON condition.conditionCode = GemMapping.icd9cm
    WHERE condition.conditionSystem LIKE '%icd9'
""")
icd9_df.write.format("iceberg").mode("append").save("local.silver.condition_standard")

In [ ]:
spark.sql("""
select
    conditionID, encounterID, patientID,
    conditionCode
from local.silver.condition_standard
""").show(10, truncate = False)

### Risk Categorization
**Risk categorization based on the Charlson Comorbidity Index (CCI)**
| Condition                             | Weight | Risk Level |
|---------------------------------------|--------|------------|
| Peripheral vascular disease           | 1      | Low        |
| Cerebrovascular disease               | 1      | Low        |
| Chronic pulmonary disease             | 1      | Low        |
| Congestive heart failure              | 1      | Low        |
| Rheumatic disease                     | 1      | Low        |
| Diabetes without chronic complication | 1      | Low        |
| Mild liver disease                    | 1      | Low        |
| Peptic ulcer disease                  | 1      | Low        |
| Renal disease                         | 1      | Low        |
| Myocardial infarction                 | 2      | Moderate   |
| Hemiplegia or paraplegia              | 2      | Moderate   |
| Malignancy                            | 2      | Moderate   |
| Diabetes with chronic complication    | 2      | Moderate   |
| Moderate or severe liver disease      | 3      | High       |
| Metastatic solid tumour               | 6      | High       |
| AIDS/HIV                              | 6      | High       |

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS local.silver.condition_risk (
    conditionId         STRING,
    encounterId         STRING,
    patientId           STRING,
    conditionCode       STRING,
    charlsonCategory    STRING,
    riskWeight          INT,
    riskLevel           STRING
) USING iceberg;
""")

In [4]:
condition_risk_df = spark.sql("""
with charlsonCategoryCondition as (
select
    conditionID,
    encounterID, patientID,
    conditionCode,
    CASE
        WHEN substring(conditionCode, 0, 4) IN ("1252")
            OR substring(conditionCode, 0, 4) like 'I21%'
            OR substring(conditionCode, 0, 4) like 'I22%'
        THEN 'Myocardial infarction'
        WHEN substring(conditionCode, 0, 4) IN ("I099","I110","I130","I132","I255","I420","I425","I426","I427","I428","I429","P290")
            OR substring(conditionCode, 0, 4) like 'I43%'
            OR substring(conditionCode, 0, 4) like 'I50%'
        THEN 'Congestive heart failure'
        WHEN substring(conditionCode, 0, 4) IN ("I731","I738","I739","I771","I790","I792","K551","K558","K559","Z958","Z959")
            OR substring(conditionCode, 0, 4) like 'I70%'
            OR substring(conditionCode, 0, 4) like 'I71%'
        THEN 'Peripheral vascular disease'
        WHEN substring(conditionCode, 0, 4) IN ("F051","G311")
            OR substring(conditionCode, 0, 4) like "F00%"
            OR substring(conditionCode, 0, 4) like "F01%"
            OR substring(conditionCode, 0, 4) like "F02%"
            OR substring(conditionCode, 0, 4) like "F03%"
            OR substring(conditionCode, 0, 4) like "G30%"
        THEN 'Cerebrovascular disease'
        WHEN substring(conditionCode, 0, 4) IN ("I278","I279","J684","J701","J703")
            OR substring(conditionCode, 0, 4) like "J40%"
            OR substring(conditionCode, 0, 4) like "J41%"
            OR substring(conditionCode, 0, 4) like "J42%"
            OR substring(conditionCode, 0, 4) like "J43%"
            OR substring(conditionCode, 0, 4) like "J44%"
            OR substring(conditionCode, 0, 4) like "J45%"
            OR substring(conditionCode, 0, 4) like "J46%"
            OR substring(conditionCode, 0, 4) like "J47%"
            OR substring(conditionCode, 0, 4) like "J60%"
            OR substring(conditionCode, 0, 4) like "J61%"
            OR substring(conditionCode, 0, 4) like "J62%"
            OR substring(conditionCode, 0, 4) like "J63%"
            OR substring(conditionCode, 0, 4) like "J64%"
            OR substring(conditionCode, 0, 4) like "J65%"
            OR substring(conditionCode, 0, 4) like "J66%"
            OR substring(conditionCode, 0, 4) like "J67%"
        THEN 'Chronic pulmonary disease'
        WHEN substring(conditionCode, 0, 4) IN ("M315", "M351","M353","M360")
            OR substring(conditionCode, 0, 4) like "M05%"
            OR substring(conditionCode, 0, 4) like "M06%"
            OR substring(conditionCode, 0, 4) like "M32%"
            OR substring(conditionCode, 0, 4) like "M32%"
            OR substring(conditionCode, 0, 4) like "M34%"
        THEN "Rheumatic disease"
        WHEN substring(conditionCode, 0, 4) like "K25%"
            OR substring(conditionCode, 0, 4) like "K26%"
            OR substring(conditionCode, 0, 4) like "K27%"
            OR substring(conditionCode, 0, 4) like "K28%"
        THEN "Peptic ulcer disease"
        WHEN substring(conditionCode, 0, 4) IN ("K700","K701","K702","K703","K709","K713","K714","K715","K717","K760","K762","K763","K764","K768","K769","Z944")
            OR substring(conditionCode, 0, 4) like 'B18%'
            OR substring(conditionCode, 0, 4) like 'K73%'
            OR substring(conditionCode, 0, 4) like 'K74%'
        THEN 'Mild liver disease'
        WHEN substring(conditionCode, 0, 4) IN ("E100","E101","E106","E108","E109","E110","E111","E116","E118","E119","E120","E121","E126","E128","E129","E130","E131","E136","E138","E139","E140","E141","E146","E148","E149")
        THEN 'Diabetes without chronic complication'
        WHEN substring(conditionCode, 0, 4) IN ("E102","E103","E104","E105","E107","E112","E113","E114","E115","E117","E122","E123","E124","E125","E127","E132","E133","E134","E135","E137","E142","E143","E144","E145","E147")
        THEN 'Diabetes with chronic complication'
        WHEN substring(conditionCode, 0, 4) IN ("G041","G114","G801","G802","G830","G831","G832","G833","G834","G839")
            OR substring(conditionCode, 0, 4) like 'G81%'
            OR substring(conditionCode, 0, 4) like 'G82%'
        THEN 'Hemiplegia or paraplegia'
        WHEN substring(conditionCode, 0, 4) IN ("I120","I131","N032","N033","N034","N035","N036","N037","N052","N053","N054","N055","N056","N057","N250","Z490","Z491","Z492","Z940","Z992")
            OR substring(conditionCode, 0, 4) like 'N18%'
            OR substring(conditionCode, 0, 4) like 'N19%'
        THEN 'Renal disease'
        WHEN substring(conditionCode, 0, 4) like "C0%"
            OR substring(conditionCode, 0, 4) like "C1%"
            OR substring(conditionCode, 0, 4) like "C20%"
            OR substring(conditionCode, 0, 4) like "C21%"
            OR substring(conditionCode, 0, 4) like "C22%"
            OR substring(conditionCode, 0, 4) like "C23%"
            OR substring(conditionCode, 0, 4) like "C24%"
            OR substring(conditionCode, 0, 4) like "C25%"
            OR substring(conditionCode, 0, 4) like "C26%"
            OR substring(conditionCode, 0, 4) like "C30%"
            OR substring(conditionCode, 0, 4) like "C31%"
            OR substring(conditionCode, 0, 4) like "C32%"
            OR substring(conditionCode, 0, 4) like "C33%"
            OR substring(conditionCode, 0, 4) like "C34%"
            OR substring(conditionCode, 0, 4) like "C37%"
            OR substring(conditionCode, 0, 4) like "C38%"
            OR substring(conditionCode, 0, 4) like "C39%"
            OR substring(conditionCode, 0, 4) like "C40%"
            OR substring(conditionCode, 0, 4) like "C41%"
            OR substring(conditionCode, 0, 4) like "C43%"
            OR substring(conditionCode, 0, 4) like "C45%"
            OR substring(conditionCode, 0, 4) like "C46%"
            OR substring(conditionCode, 0, 4) like "C47%"
            OR substring(conditionCode, 0, 4) like "C48%"
            OR substring(conditionCode, 0, 4) like "C49%"
            OR substring(conditionCode, 0, 4) like "C50%"
            OR substring(conditionCode, 0, 4) like "C51%"
            OR substring(conditionCode, 0, 4) like "C52%"
            OR substring(conditionCode, 0, 4) like "C53%"
            OR substring(conditionCode, 0, 4) like "C54%"
            OR substring(conditionCode, 0, 4) like "C55%"
            OR substring(conditionCode, 0, 4) like "C56%"
            OR substring(conditionCode, 0, 4) like "C57%"
            OR substring(conditionCode, 0, 4) like "C58%"
            OR substring(conditionCode, 0, 4) like "C60%"
            OR substring(conditionCode, 0, 4) like "C61%"
            OR substring(conditionCode, 0, 4) like "C62%"
            OR substring(conditionCode, 0, 4) like "C63%"
            OR substring(conditionCode, 0, 4) like "C64%"
            OR substring(conditionCode, 0, 4) like "C65%"
            OR substring(conditionCode, 0, 4) like "C66%"
            OR substring(conditionCode, 0, 4) like "C67%"
            OR substring(conditionCode, 0, 4) like "C68%"
            OR substring(conditionCode, 0, 4) like "C69%"
            OR substring(conditionCode, 0, 4) like "C70%"
            OR substring(conditionCode, 0, 4) like "C71%"
            OR substring(conditionCode, 0, 4) like "C72%"
            OR substring(conditionCode, 0, 4) like "C73%"
            OR substring(conditionCode, 0, 4) like "C74%"
            OR substring(conditionCode, 0, 4) like "C75%"
            OR substring(conditionCode, 0, 4) like "C76%"
            OR substring(conditionCode, 0, 4) like "C81%"
            OR substring(conditionCode, 0, 4) like "C82%"
            OR substring(conditionCode, 0, 4) like "C83%"
            OR substring(conditionCode, 0, 4) like "C84%"
            OR substring(conditionCode, 0, 4) like "C85%"
            OR substring(conditionCode, 0, 4) like "C88%"
            OR substring(conditionCode, 0, 4) like "C90%"
            OR substring(conditionCode, 0, 4) like "C91%"
            OR substring(conditionCode, 0, 4) like "C92%"
            OR substring(conditionCode, 0, 4) like "C93%"
            OR substring(conditionCode, 0, 4) like "C94%"
            OR substring(conditionCode, 0, 4) like "C95%"
            OR substring(conditionCode, 0, 4) like "C96%"
            OR substring(conditionCode, 0, 4) like "C97%"
        THEN 'Malignancy'
        WHEN substring(conditionCode, 0, 4) IN ("I850","I859","I864","I982","K704","K711","K721","K729","K765","K766","K767")
        THEN 'Moderate or severe liver disease'
        WHEN substring(conditionCode, 0, 4) like 'C77%'
            OR substring(conditionCode, 0, 4) like 'C78%'
            OR substring(conditionCode, 0, 4) like 'C79%'
            OR substring(conditionCode, 0, 4) like 'C80%'
        THEN "Metastatic solid tumour"
        WHEN substring(conditionCode, 0, 4) like 'B20%'
            OR substring(conditionCode, 0, 4) like 'B21%'
            OR substring(conditionCode, 0, 4) like 'B22%'
            OR substring(conditionCode, 0, 4) like 'B24%'
        THEN "AIDS/HIV"
    END AS charlsonCategory
from local.silver.condition_standard
)
select
    conditionID,
    encounterID, patientID,
    conditionCode, charlsonCategory,
    CASE
        WHEN charlsonCategory IN ("Peripheral vascular disease", "Cerebrovascular disease", "Chronic pulmonary disease", "Congestive heart failure", "Rheumatic disease", "Diabetes without chronic complication", "Mild liver disease", "Peptic ulcer disease", "Renal disease")
        THEN 1
        WHEN charlsonCategory IN ("Myocardial infarction", "Hemiplegia or paraplegia", "Malignancy", "Diabetes with chronic complication")
        THEN 2
        WHEN charlsonCategory IN ("Moderate or severe liver disease")
        THEN 3
        WHEN charlsonCategory IN ("Metastatic solid tumour", "AIDS/HIV")
        THEN 6
        ELSE 0
    END riskWeight,
    CASE
        WHEN charlsonCategory IN ("Peripheral vascular disease", "Cerebrovascular disease", "Chronic pulmonary disease", "Congestive heart failure", "Rheumatic disease", "Diabetes without chronic complication", "Mild liver disease", "Peptic ulcer disease", "Renal disease")
        THEN "Low"
        WHEN charlsonCategory IN ("Myocardial infarction", "Hemiplegia or paraplegia", "Malignancy", "Diabetes with chronic complication")
        THEN "Moderate"
        WHEN charlsonCategory IN ("Moderate or severe liver disease", "Metastatic solid tumour", "AIDS/HIV")
        THEN "High"
        ELSE charlsonCategory
    END riskLevel
from charlsonCategoryCondition;
""")

condition_risk_df.write.format("iceberg").mode("overwrite").save("local.silver.condition_risk")

In [ ]:
spark.sql("select charlsonCategory, riskLevel, count(*) from local.silver.condition_risk group by 1,2").show(20, truncate = False)

### Case-Based Aggregation

In [ ]:
spark.sql("""
CREATE TABLE IF NOT EXISTS local.silver.condition_rollup  (
    encounterId           STRING,
    patientID             STRING,
    riskScore             INT,
    highRiskCondition     INT,
    moderateRiskCondition INT,
    lowRiskCondition      INT
) USING iceberg;
""")

In [30]:
condition_rollup_df = spark.sql("""
SELECT 
    encounterId,
    patientID,
    SUM(riskWeight) riskScore,
    SUM(CASE WHEN riskLevel = 'High' THEN 1 ELSE 0 END) AS highRiskCondition,
    SUM(CASE WHEN riskLevel = 'Moderate' THEN 1 ELSE 0 END) AS moderateRiskCondition,
    SUM(CASE WHEN riskLevel = 'Low' THEN 1 ELSE 0 END) AS lowRiskCondition
FROM local.silver.condition_risk
GROUP BY encounterId, patientID;
""")
condition_rollup_df.write.format("iceberg").mode("overwrite").save("local.silver.condition_rollup")

In [ ]:
condition_rollup_df.show(10, truncate = False)

## Encounter Gold

In [ ]:
spark.sql("""
    SELECT
        encounter.encounterId,
        encounter.patientId,
        patient.gender,
        CASE 
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 18 AND 29 THEN 'young adults'
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 30 AND 39 THEN 'young adulthood'
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 40 AND 49 THEN 'early-middle age'
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 50 AND 59 THEN 'late-middle age'
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 60 AND 69 THEN 'mid-old age'
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 70 AND 79 THEN 'senior-old age'
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 80 AND 89 THEN 'very senior-old'
            WHEN date_diff(year, patient.birthDate, encounter.periodStart) BETWEEN 90 AND 115 THEN 'centenarians'
            ELSE 'other age groups'
        END AS ageGroup,
        patient.maritalStatus,
        encounter.duration stayLength,
        encounter.status encounterStatus,
        encounter.encounterClass,
        encounter.admitSource,
        encounter.dischargeDisposition,
        encounter.displayType encounterType,
        encounter.priority,
        IFNULL(condition.riskScore, 0) riskScore,
        IFNULL(condition.highRiskCondition, 0) highRiskCondition,
        IFNULL(condition.moderateRiskCondition, 0) moderateRiskCondition,
        IFNULL(condition.lowRiskCondition, 0) lowRiskCondition,
        encounter.readmissionStatus
    FROM local.silver.encounter
    LEFT JOIN local.silver.patient
        ON encounter.patientId = patient.patientID
    LEFT JOIN local.silver.condition_rollup condition
        ON encounter.encounterId = condition.encounterId
        AND encounter.patientID = condition.patientID
""").show(30, truncate=False)